### Type I for regular and weighted (NA zeroed out) coefficient estimation

- First try to optimize for regular estimation procedure

In [6]:
from class_FE_regression import *

In [7]:
N = 30
T = 15
beta = [0, 0]
K = len(beta)
min_var, max_var = 0, 2
shrink_factor = 0 # set > 0 to generate cross-sectional correlation of size max_var/shrink_factor
range_theta = [0,10]
reps = int(1e4)                              

sim1 = panel_simulation(N, T, beta, reps, min_var, max_var, range_theta, shrink_factor)
sim1.simulate_Y()

est_unweighted = panel_estimation(sim1.X_observed, sim1.Y, N, T, method = 'FE_OLS')
est_unweighted.estimate_coefs()


In [8]:
est_unweighted.calc_t_values()
est_unweighted.rejection_rate()
est_unweighted.proportion_H0_rejected

array([0.0578, 0.0576])

### Weighted FE OLS
- Now, do the adjusted version for the weighted FE OLS.
    - Need df adjustment to exclude the missing observations in df calculation

In [9]:
# Simulate removing missing datapoints
# Randomly select missing_prob amount of indices to set to NA for X
missing_prob = 0.1
missing_indices_X = np.random.rand(*sim1.X.shape) < missing_prob  # Define a probability for missing data

sim1.X_observed[missing_indices_X] = np.nan

est_weighted = panel_estimation(sim1.X_observed, sim1.Y, N, T, method = 'weighted_FE_OLS')
est_weighted.estimate_coefs()

In [10]:
est_weighted.calc_t_values()
est_weighted.rejection_rate()
est_unweighted.proportion_H0_rejected

array([0.0578, 0.0576])

- Compare type I error (rejection rate) in weighted vs unweighted

In [13]:
# Compare 
print(f"Unweighted Type I error: {est_unweighted.proportion_H0_rejected}\nWeighted Type I error: {est_weighted.proportion_H0_rejected}")

Unweighted Type I error: [0.0604 0.0588]
Weighted Type I error: [0.0565 0.0554]


---------------------------
- Type I error function

In [11]:
def type_I_error_revamped():    

    # Tensor implementation of Type I errors
    ## First calculates the t-values for all the coefficients
    ## Then calculates whether the coefficient rejects H0 of beta=0
    ## Since true beta equals 0, it should not reject H0, hence computing the type I errors
    ### Based on Pesaran p.642,643

    # Compute residuals
    H0 = np.zeros(K)
    # resids_all = est1.M_dot_Y - np.einsum('hntr, r -> hnt', est1.M_dot_X, np.zeros(2))
    resids_all = est1.M_dot_Y - np.einsum('hntr, r -> hnt', est1.M_dot_X, H0)

    # Compute variance for each N
    variances_all = np.sum(resids_all ** 2, axis = 2) / T

    # Construct Gamma matrices, is RepsxNxTxT
    ## Each TxT matrix has the corresponding variance on the diagonal
    identity_matrices = np.tile(np.eye(T), (reps, N, 1, 1))
    Gamma = identity_matrices * variances_all[:, :, np.newaxis, np.newaxis]

    V_FENT = np.einsum('anti,antj,antk->aik', est1.M_dot_X, Gamma, est1.M_dot_X)
    Omega = np.einsum('aij,ajk,akl->ail', est1.Q_inv, V_FENT, est1.Q_inv)

    # Compute the square root of the diagonal elements of Omega
    diagonal_sqrt_Omega = np.sqrt(np.diagonal(Omega, axis1=1, axis2=2))

    # Divide the coefficients by the square root of the diagonal elements
    t_vals = est1.coefficients / diagonal_sqrt_Omega

    # Calculate degrees of freedom and critical value
    df = N * T - 2 - N  # degrees of freedom
    critical_value = scipy.stats.t.ppf(1 - 0.05 / 2, df)

    # Calculate absolute t-values
    abs_t_vals = np.abs(t_vals)

    # Count the number of rejections for beta1 and beta2
    nr_of_rejections_beta = []

    for k in range(K):
        nr_of_rejections_beta.append(np.sum(abs_t_vals[:, k] > critical_value))

    proportion_H0_rejected = np.array(nr_of_rejections_beta) / reps

    print(proportion_H0_rejected)

    

In [7]:
def type_I_error():    
    t_vals = []
    M = np.eye(T) - (1/T) * np.ones((T, T))


    for rep in range(reps):
        X = M @ sim1.X_observed[rep]
        y = sim1.Y[rep] @ M
        beta = est1.coefficients[rep]
        
        resids = y - ((X @ beta))
        
        variances = np.sum(resids ** 2, axis = 1) / (T)
        
        V_FENT = np.zeros((2,2))
        Q_FENT = np.zeros((2,2))

        for i in range(N):
            Gamma_i = np.eye(T) * variances[i]
            V_FENT += X[i].T @ M @ Gamma_i @ M @ X[i]
            Q_FENT += X[i].T @ M @ X[i]
            
        Q_inv = est1.Q_inv[rep]
        Omega = Q_inv @ V_FENT @ Q_inv
        t_vals.append(est1.coefficients[rep] / np.sqrt(np.diag(Omega)))


    df = N*T - 2 - N# degrees of freedom
    critical_value = scipy.stats.t.ppf(1 - 0.05 / 2, df)

    rejections_beta1 = 0
    rejections_beta2 = 0

    for t_val in t_vals:
        if abs(t_val[0]) > critical_value:
            rejections_beta1 += 1
        if abs(t_val[1]) > critical_value:
            rejections_beta2 += 1

    print(rejections_beta1/reps, rejections_beta2/reps)

In [ ]:
N = 30
T = 15
beta = [0, 0]
K = len(beta)
min_var, max_var = 0, 2
shrink_factor = 0 # set > 0 to generate cross-sectional correlation of size max_var/shrink_factor
range_theta = [0,10]
reps = int(1e4)                              

sim1 = panel_simulation(N, T, beta, reps, min_var, max_var, range_theta, shrink_factor)
sim1.simulate_Y()

est_unweighted = panel_estimation(sim1.X_observed, sim1.Y, N, T, method = 'FE_OLS')
est_unweighted.estimate_coefs()

betaa = np.array(beta)

- Speed comparison of old (slow) implementation vs optimized (with tensor contractions)

In [ ]:
type_I_error()

In [ ]:
type_I_error_revamped()